In [53]:
# installing dependencies 

from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D, SeparableConv2D
from keras.utils import np_utils
from random import randint
import numpy as np
import os
import PIL
from PIL import Image, ImageTk, ImageDraw
from tkinter import *

In [ ]:
### for storing google colab dataset for 10 categories

!mkdir data

In [ ]:
cd data

/content/data


In [ ]:
### downloading numpy bitmaps of 10 categories from google storage

!wget https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/apple.npy
!wget https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/banana.npy
!wget https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/crab.npy
!wget https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/crown.npy
!wget https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/dog.npy
!wget https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/duck.npy
!wget https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/eye.npy
!wget https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/feather.npy
!wget https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/tree.npy
!wget https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/pizza.npy

--2022-01-19 04:31:51--  https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/apple.npy
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.212.128, 173.194.213.128, 173.194.215.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.212.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 113462128 (108M) [application/octet-stream]
Saving to: ‘apple.npy’

apple.npy           100%[===================>] 108.21M  55.3MB/s    in 2.0s    

2022-01-19 04:31:53 (55.3 MB/s) - ‘apple.npy’ saved [113462128/113462128]

--2022-01-19 04:31:53--  https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/banana.npy
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.212.128, 173.194.213.128, 173.194.215.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.212.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 241421904 (230M) [application/octet

In [ ]:
### checking each file

ls

apple.npy   crab.npy   dog.npy   eye.npy      pizza.npy
banana.npy  crown.npy  duck.npy  feather.npy  tree.npy


In [43]:
cd ..

/content


In [54]:
# variable initialization

N_OBJ = 10
OBJ = {0: "Apple", 1: "Banana", 2: "Crab", 3: "Crown", 4: "Dog", 5: "Duck", 6: "Eye", 7: "Feather", 8: "Pizza", 9: "Tree"}

# number of samples to take in each category
N = 2000

# 15 iterations for model training
N_EPOCHS = 15

# data files in the same order as defined in our data
files = ["apple.npy", "banana.npy", "crab.npy", "crown.npy", "dog.npy", "duck.npy", "eye.npy", "feather.npy", "pizza.npy", "tree.npy"]

In [55]:
def load(dir, reshaped, files):
    "Load .npy files and return them as numpy arrays. \
    Takes in a list of filenames and returns a list of numpy arrays."

    data = []
    for file in files:
        f = np.load(dir + file)
        if reshaped:
            new_f = []
            for i in range(len(f)):
                x = np.reshape(f[i], (28, 28))
                x = np.expand_dims(x, axis=0)
                x = np.reshape(f[i], (28, 28, 1))
                new_f.append(x)
            f = new_f
        data.append(f)
    return data


def normalize(data):
    "Takes a list or a list of lists and returns its normalized form"

    return np.interp(data, [0, 255], [-1, 1])


def denormalize(data):
    "Takes a list or a list of lists and returns its denormalized form"

    return np.interp(data, [-1, 1], [0, 255])


def visualize(array):
    "Visulaze a 2D array as an Image"
    array = np.reshape(array, (28,28))
    img = Image.fromarray(array)
    return img


def set_limit(arrays, n):
    "Limit elements from each array up to n elements and return a single list"
    new = []
    for array in arrays:
        i = 0
        for item in array:
            if i == n:
                break
            new.append(item)
            i += 1
    return new


def make_labels(N1, N2):
    "make labels from 0 to N1, each repeated N2 times"
    labels = []
    for i in range(N1):
        labels += [i] * N2
    return labels

In [56]:

#second argument is True for reshaping the image to a 28x28 form. A conv net expects this format.
objects = load("data/", True, files)
 



# limit no of samples in each class to N
objects = set_limit(objects, N)

# normalize the values
objects = list(map(normalize, objects))

# define the labels
labels = make_labels(N_OBJ, N)

# prepare the data with 90%-10% split for training-testing
x_train, x_test, y_train, y_test = train_test_split(objects, labels, test_size=0.2)

# one hot encoding
Y_train = np_utils.to_categorical(y_train, N_OBJ)
Y_test = np_utils.to_categorical(y_test, N_OBJ)


In [57]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28,28,1)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(N_OBJ, activation='softmax'))

In [58]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


# train
model.fit(np.array(x_train), np.array(Y_train), batch_size=32, epochs=N_EPOCHS)

print("Training complete")

print("Evaluating model")
preds = model.predict(np.array(x_test))

score = 0
for i in range(len(preds)):
    if np.argmax(preds[i]) == y_test[i]:
        score += 1

print("Accuracy: ", ((score + 0.0) / len(preds)) * 100)


model.save("object_recog"+ ".h5")
print("Model saved")


Epoch 1/15
500/500 [==============================] - 46s 90ms/step - loss: 0.8432 - accuracy: 0.7336
Epoch 2/15
500/500 [==============================] - 46s 92ms/step - loss: 0.5096 - accuracy: 0.8433
Epoch 3/15
500/500 [==============================] - 45s 91ms/step - loss: 0.4022 - accuracy: 0.8759
Epoch 4/15
500/500 [==============================] - 47s 94ms/step - loss: 0.3275 - accuracy: 0.8959
Epoch 5/15
500/500 [==============================] - 44s 89ms/step - loss: 0.2865 - accuracy: 0.9090
Epoch 6/15
500/500 [==============================] - 44s 87ms/step - loss: 0.2413 - accuracy: 0.9189
Epoch 7/15
500/500 [==============================] - 45s 90ms/step - loss: 0.2046 - accuracy: 0.9326
Epoch 8/15
500/500 [==============================] - 45s 90ms/step - loss: 0.1941 - accuracy: 0.9341
Epoch 9/15
500/500 [==============================] - 45s 90ms/step - loss: 0.1661 - accuracy: 0.9434
Epoch 10/15
500/500 [==============================] - 45s 90ms/step - loss: 0.152